In [1]:
import dace
import cupy as cp
import numpy as np
from IPython.display import Code
from typing import Optional

from dace import SDFG, properties
from dace.config import Config
from dace.transformation import pass_pipeline as ppl, transformation
from dace.sdfg import nodes
from dace import dtypes
from dace.transformation.passes.gpustream_scheduling import NaiveGPUStreamScheduler


In [2]:
@dace.program
def test1(A: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global,
         B: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global,
         C: dace.uint32[10],
         D: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global
         ):
    A[:] = B[:]
    C[:] = D[:]


@dace.program
def test2(A: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global,
         B: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global,
         C: dace.uint32[10],
         D: dace.uint32[10] @ dace.dtypes.StorageType.GPU_Global
         ):
    
    for i in dace.map[0:10] @ dace.dtypes.ScheduleType.GPU_Device:
        A[i] = B[i]
    
    for j in dace.map[0:10] @ dace.dtypes.ScheduleType.GPU_Device:
        C[j] = D[j]

sdfg = test2.to_sdfg()
sdfg

SDFG (test2)

In [3]:

gpu_stream_access_template = "__state->gpu_context->streams[{gpu_stream}]"  

# Initialize and configure GPU stream scheduling pass
gpu_stream_pass = NaiveGPUStreamScheduler()
gpu_stream_pass.set_gpu_stream_access_template(gpu_stream_access_template)
assigned_streams = gpu_stream_pass.apply_pass(sdfg, None)
sdfg

SDFG (test2)